In [28]:
import json
import re
import pandas as pd

In [2]:
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    df = df.drop([col for col in df.columns if '_step' in col], axis=1) # drop training step number columns
    df = df.replace('undefined', 0.0)
    df = df.astype('float64')
    
    return df

df = load_csv('../reports/thesis/tables/top_10_test_rouge_l_f1.csv')
df.head()

,epoch,rs_1_09 - test_rouge_L_f1,rs_1_08 - test_rouge_L_f1,rs_1_13 - test_rouge_L_f1,rs_1_12 - test_rouge_L_f1,rs_1_11 - test_rouge_L_f1,rs_1_10 - test_rouge_L_f1,rs_1_07 - test_rouge_L_f1,rs_1_06 - test_rouge_L_f1,rs_1_05 - test_rouge_L_f1
0,1.0,0.248835,0.299247,0.424240,0.164319,0.478707,0.396162,0.192241,0.411260,0.177816
1,2.0,0.298319,0.426575,0.486871,0.186104,0.523838,0.503040,0.254408,0.451513,0.219013
2,3.0,0.318446,0.493871,0.510211,0.205533,0.513825,0.535803,0.306846,0.463835,0.264076
3,4.0,0.324207,0.515968,0.521742,0.217778,0.522579,0.552802,0.353147,0.470170,0.316740
4,5.0,0.263045,0.526855,0.497633,0.233040,0.534139,0.562303,0.000000,0.470495,0.358042


In [34]:
def get_best_score(df, key, metric_name):
    best_row = df.loc[df[key].idxmax()][['epoch', key]]
    experiment_id = int(re.search('rs_1_(.*) - test_rouge_L_f1', key, re.IGNORECASE).group(1))
    return pd.DataFrame(data={'experiment_id': [f'experiment_{experiment_id}'], 'epoch': [best_row['epoch']], metric_name: [best_row[key]]})

get_best_score(df, 'rs_1_10 - test_rouge_L_f1', 'test_rouge_L_f1')

,experiment_id,epoch,test_rouge_L_f1
0,experiment_10,10.0,0.58293


In [35]:
def get_best_scores(df, metric='test_rouge_L_f1'):
    return pd.concat([
        get_best_score(df, column, metric_name=metric)
        for column in df.columns
        if metric in column
    ]).sort_values(by=metric, ascending=False).set_index('experiment_id').astype({'epoch': 'int64'})

df_scores = get_best_scores(df)
df_scores

,epoch,test_rouge_L_f1
experiment_id,,
experiment_10,10,0.582930
experiment_8,28,0.566747
experiment_5,40,0.565396
experiment_11,11,0.560674
experiment_13,21,0.555791
experiment_6,6,0.476592
experiment_12,30,0.437038
experiment_7,4,0.353147
experiment_9,4,0.324207


In [39]:
df_experiment_params = pd.read_json('../experiments/random_search_1.json').transpose()
df_experiment_params

,batch_size,dropout_rate,embedding_dim,latent_dim,learning_rate,max_input_seq_length,max_output_seq_length,vocabulary_size
experiment_0,512.0,0.118569,32.0,640.0,0.000192,75.0,8.0,7000.0
experiment_1,128.0,0.087517,128.0,256.0,0.000118,100.0,6.0,2000.0
experiment_2,64.0,0.185119,112.0,256.0,0.006695,125.0,5.0,5000.0
experiment_3,1024.0,0.155631,32.0,512.0,0.001129,75.0,8.0,7000.0
experiment_4,256.0,0.135776,32.0,384.0,0.005168,50.0,8.0,4000.0
experiment_5,512.0,0.094720,128.0,640.0,0.000336,150.0,7.0,2000.0
experiment_6,32.0,0.064828,112.0,1024.0,0.003592,75.0,8.0,6000.0
experiment_7,64.0,0.089990,32.0,384.0,0.000402,75.0,6.0,5000.0
experiment_8,256.0,0.042077,80.0,256.0,0.002340,175.0,5.0,3000.0
experiment_9,128.0,0.087720,96.0,1024.0,0.006250,150.0,8.0,6000.0


In [47]:
df_experiment_params.join(df_scores).dropna().round(4).astype('object').sort_values(by='test_rouge_L_f1', ascending=False).transpose()

,experiment_10,experiment_8,experiment_5,experiment_11,experiment_13,experiment_6,experiment_12,experiment_7,experiment_9
batch_size,512,256,512,32,32,32,64,64,128
dropout_rate,0.1164,0.0421,0.0947,0.0276,0.121,0.0648,0.1676,0.09,0.0877
embedding_dim,128,80,128,48,48,112,64,32,96
latent_dim,768,256,640,640,256,1024,256,384,1024
learning_rate,0.0032,0.0023,0.0003,0.0018,0.0013,0.0036,0.0001,0.0004,0.0062
max_input_seq_length,175,175,150,175,125,75,100,75,150
max_output_seq_length,6,5,7,7,7,8,7,6,8
vocabulary_size,7000,3000,2000,2000,5000,6000,7000,5000,6000
epoch,10,28,40,11,21,6,30,4,4
test_rouge_L_f1,0.5829,0.5667,0.5654,0.5607,0.5558,0.4766,0.437,0.3531,0.3242


In [27]:
df_scores.transpose().round(4)

experiment_id,10,8,5,11,13,6,12,7,9
epoch,10.0000,28.0000,40.0000,11.0000,21.0000,6.0000,30.000,4.0000,4.0000
test_rouge_L_f1,0.5829,0.5667,0.5654,0.5607,0.5558,0.4766,0.437,0.3531,0.3242
